In [4]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
mpl.rcParams['figure.figsize'] = (12,10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [14]:
# read data set

file = tf.keras.utils
raw_df = pd.read_csv('https://raw.githubusercontent.com/tjsipin/MacDonald-REU-Summer-22/main/models/data/aad.csv')
raw_df.head()

,Code,Year,Country,Name,Population,Chikungunya,Cutaneous.Leishmaniasis,Dengue,Malaria,Mucosal.Leishmaniasis,...,Savanna_Riverlakeocean,Floodforest_Wetland,Floodforest_Farming,Floodforest_Nonveg,Floodforest_Riverlakeocean,Wetland_Farming,Wetland_Nonveg,Farming_Nonveg,Farming_Riverlakeocean,Nonveg_Riverlakeocean
0,5001,2000,Colombia,MEDELLIN,1541212,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5001,2001,Colombia,MEDELLIN,1572973,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5001,2002,Colombia,MEDELLIN,1609676,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5001,2003,Colombia,MEDELLIN,1655464,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5001,2004,Colombia,MEDELLIN,1693608,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# summary of raw_df

raw_df.describe()

,Code,Year,Population,Chikungunya,Cutaneous.Leishmaniasis,Dengue,Malaria,Mucosal.Leishmaniasis,Visceral.Leishmaniasis,Yellow.Fever,...,Savanna_Riverlakeocean,Floodforest_Wetland,Floodforest_Farming,Floodforest_Nonveg,Floodforest_Riverlakeocean,Wetland_Farming,Wetland_Nonveg,Farming_Nonveg,Farming_Riverlakeocean,Nonveg_Riverlakeocean
count,1.711400e+05,171140.000000,1.711400e+05,13845.000000,131202.000000,125638.000000,136766.000000,31050.000000,112472.000000,100224.000000,...,38862.000000,3.886200e+04,38862.000000,38862.000000,38862.000000,38862.000000,38862.000000,38862.000000,38862.000000,38862.000000
mean,2.143807e+06,2009.500000,2.682784e+04,0.246878,0.343305,2.583887,1.634192,0.019548,0.022124,0.000256,...,359.432582,3.061520e+03,95.940147,1.703927,84.505584,114.306958,41.903402,3125.659462,1506.249318,736.545340
std,1.708727e+06,5.766298,1.627907e+05,2.654742,1.510583,8.493871,18.866805,0.147743,0.094924,0.014002,...,1908.244556,3.197677e+04,889.582216,17.309983,776.102713,1865.146088,224.636149,7115.279714,4838.867423,2423.235964
min,5.001000e+03,2000.000000,9.800000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.103030e+05,2004.750000,3.735000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,12.000000,0.000000
50%,2.503555e+06,2009.500000,8.277000e+03,0.000000,0.000000,0.124899,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000,129.000000,61.000000
75%,3.514403e+06,2014.250000,1.849100e+04,0.000000,0.112999,1.299275,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2913.750000,816.000000,459.000000
max,5.300108e+06,2019.000000,1.035358e+07,172.139303,91.746021,379.199864,1535.365979,5.425631,5.350195,2.088409,...,52408.000000,1.676670e+06,30657.000000,692.000000,23942.000000,121813.000000,6451.000000,148015.000000,200322.000000,96656.000000


In [ ]:
# examine class label imbalance